displacement current
D and E

```
% diff is exteriori derivative or some kind of geometric derrivative
diff(d) = rho
diff(b) = zero
diff(h) = 
diff(e)

```

https://en.wikipedia.org/wiki/Flux_tube
"Counting" flux lines / tubes. An interesting integration variation. Set flux size. Relax boundaries to get balanced/equal flux.
Or set number of tubes (well, if we know the tube source, kind of same thing.)

Riemann
Riemann stieljes
Lebesgue - find size of level sets.



The charge in a resistive medium. The continuous RC problem


Charge on wire kinks. Is that real?
Fringe field. Dielectric insertion. That's an insane cheat.

Idealizations and paradoxes.
Some tempting idealizations make you want to put this or that out at ininifty and not care. Some conclusions are paradoxical. An effect becomes small as something geometrically becomes large

A magnetic monopole can be made by a long solenoid.


Dimensionality reductions



magnetohydrodynamiucs https://en.wikipedia.org/wiki/Magnetohydrodynamics
There were some kind domains where like the magnetic field gets locked in




meep https://meep.readthedocs.io/en/latest/

empy

lenzs law


charge vortex duality
coninuity in 2d is like curl in 2d



In [ ]:
from z3 import *
M = DeclareSort('M')
ScalarField = ArraySort(M, RealSort())
Vec3Field = DeclareSort('Vec3Field')
E,B = Consts('E B', Vec3Field)
rho = Const('rho', ScalarField)

div = Function('div', Vec3Field, ScalarField)
grad = Function('grad', ScalarField, Vec3Field)
curl = Function('curl', Vec3Field, Vec3Field)

vzero = Const('vzero', Vec3Field)
szero = Const("szero", ScalarField)
div(E) == szero

laplacian = Function('laplacian', ScalarField, ScalarField)
hodge = Function('hodge', Vec3Field, Vec3Field)
curl(curl(E)) = grad(div(E)) - laplacian(E)



In [6]:
import sympy as sp
x,y,z,t = sp.symbols("x y z t")
def VecField(name):
    return [sp.Function(name+"{n}")(x,y,z,t) for n in range(3)]
def curl(f):
    return [f[1].diff(z) - f[2].diff(y),
            f[2].diff(x) - f[0].diff(z),
            f[0].diff(y) - f[1].diff(x)]
E = VecField("E")
B = VecField("B")
J = VecField("J")
rho = sp.Function("rho")(x,y,z,t)
def div(f):
    return f[0].diff(x) + f[1].diff(y) + f[2].diff(z)

def cont(rho,J):
    return sp.Eq(rho.diff(t), div(J))

sp.Eq(curl(E), [B.diff(t) for Bx in B])


AttributeError: 'list' object has no attribute 'diff'

In [ ]:
# https://docs.scipy.org/doc/scipy/reference/optimize.html
# integrate some polynomials
# sin, cos, exp, log, etc.
# maybe some 

def tube_integ(f,N):
    h = 1/N
    x = 0
    for i in range(N):
        x += f(i*h)*h
    return x

In [1]:
import meep as mp
cell = mp.Vector3(16,8,0)
geometry = [mp.Block(mp.Vector3(mp.inf,1,mp.inf),
                     center=mp.Vector3(),
                     material=mp.Medium(epsilon=12))]
sources = [mp.Source(mp.ContinuousSource(frequency=0.15),
                     component=mp.Ez,
                     center=mp.Vector3(-7,0))]
pml_layers = [mp.PML(1.0)]
resolution = 10
sim = mp.Simulation(cell_size=cell,
                    boundary_layers=pml_layers,
                    geometry=geometry,
                    sources=sources,
                    resolution=resolution)
sim.run(until=200)
sim.plot2D(fields=mp.Ez)



AttributeError: module 'meep' has no attribute 'Vector3'